## Reasoning and Acting (ReAct)​

Agent to implement ReAct logic

In [3]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "ls__632ae1a420dc4b46b2894dd749f33b2e"

os.environ["TAVILY_API_KEY"] = "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"

os.environ["AZURE_OPENAI_ENDPOINT"] = "https://js-playground-01.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "43d936898d7645c3b3c1ebf294f04cd3"

In [1]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import AzureChatOpenAI

In [4]:
tools = [TavilySearchResults(max_results=1)]

In [7]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

In [8]:
# Choose the LLM to use
llm = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment="gpt4-32k",
)

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [9]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [10]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
I need to find information about LangChain.
Action: tavily_search_results_json
Action Input: "what is LangChain"[{'url': 'https://www.langchain.com/', 'content': 'Join Discord\nContribute on GitHub\n5.3M\nMonthly Downloads\n67k\nGithub Stars\n1800+\nContributors\n34k+\nApps Powered\nReady to build?\nGo To Docs\nGo To Docs\nGo To Docs\nContact Sales\nContact Sales\nContact Sales\nProducts and use-cases\nLangChain\nLangSmith\nRetrieval\nAgents\nInspiration\nCode\nGitHub\nLangChain Hub\nPython Docs\nJS/TS Docs\nSocial\nTwitter\nDiscord\nBlog\nLinkedIn\nYouTube\nTerms of Service\nSign up for our newsletter\nProducts and use-cases\nLangChain\nLangSmith\nRetrieval\nAgents\nInspiration\nCode\nGitHub\nLangChain Hub\nPython Docs\nJS/TS Docs\nSocial\nTwitter\nDiscord\nBlog\nLinkedIn\nYouTube\nTerms of Service\nSign up for our newsletter\nProducts and use-cases\nLangChain\nLangSmith\nRetrieval\nAgents\nInspiration\nCode\nGitHub\nLangChain Hub\nPython Docs\n

{'input': 'what is LangChain?',
 'output': 'LangChain is a tool for building language model applications. It provides standardized interfaces and integrations for interfacing with language models and application-specific data, making it easier to build and customize applications. It is designed to simplify the entire application lifecycle and supports the transition from prototypes to production. It is part of a broader ecosystem of tools and offers a range of features for application customization.'}

## Use chat hhistory

In [11]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react-chat")

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [12]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "input": "what's my name? Only use a tool if needed, otherwise respond with Final Answer",
        # Notice that chat_history is a string, since this prompt is aimed at LLMs, not chat models
        "chat_history": "Human: Hi! My name is Bob\nAI: Hello Bob! Nice to meet you",
    }
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Your name is Bob.

> Finished chain.


{'input': "what's my name? Only use a tool if needed, otherwise respond with Final Answer",
 'chat_history': 'Human: Hi! My name is Bob\nAI: Hello Bob! Nice to meet you',
 'output': 'Your name is Bob.'}